<a href="https://colab.research.google.com/github/zhannatoleubek-png/special-okx-chainsaw/blob/main/%D0%94%D0%B8%D0%BF%D1%81%D0%B8%D0%BA3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import requests
import warnings
import time
import concurrent.futures
from datetime import datetime, timedelta
from typing import List, Dict, Any
warnings.filterwarnings('ignore')

class OKXFuturesScanner:
    def __init__(self):
        self.base_url = "https://www.okx.com"
        self.all_symbols = []
        self.last_scan_time = None

    def get_all_usdt_futures_symbols(self) -> List[str]:
        """Получение ВСЕХ USDT фьючерсных пар с OKX"""
        try:
            print("🔍 Получение списка всех USDT фьючерсных пар...")

            all_symbols = []
            url = f"{self.base_url}/api/v5/public/instruments"
            params = {'instType': 'SWAP'}

            response = requests.get(url, params=params, timeout=10)
            data = response.json()

            if data['code'] == '0':
                for instrument in data['data']:
                    if (instrument['instId'].endswith('-SWAP') and
                        '-USDT-' in instrument['instId']):
                        all_symbols.append(instrument['instId'])

            print(f"✅ Найдено {len(all_symbols)} USDT фьючерсных пар")
            return all_symbols

        except Exception as e:
            print(f"❌ Ошибка получения списка пар: {e}")
            return self._get_popular_usdt_futures()

    def _get_popular_usdt_futures(self) -> List[str]:
        """Список популярных USDT фьючерсов по умолчанию"""
        return [
            "BTC-USDT-SWAP", "ETH-USDT-SWAP", "ADA-USDT-SWAP", "DOT-USDT-SWAP",
            "LINK-USDT-SWAP", "LTC-USDT-SWAP", "BCH-USDT-SWAP", "XRP-USDT-SWAP",
            "EOS-USDT-SWAP", "ETC-USDT-SWAP", "FIL-USDT-SWAP", "ATOM-USDT-SWAP",
            "SOL-USDT-SWAP", "DOGE-USDT-SWAP", "MATIC-USDT-SWAP", "AVAX-USDT-SWAP",
            "APT-USDT-SWAP", "ARB-USDT-SWAP", "OP-USDT-SWAP", "SUI-USDT-SWAP",
            "SEI-USDT-SWAP", "NEAR-USDT-SWAP", "ALGO-USDT-SWAP", "XLM-USDT-SWAP",
            "XMR-USDT-SWAP", "ZEC-USDT-SWAP", "DASH-USDT-SWAP", "WAVES-USDT-SWAP",
            "SAND-USDT-SWAP", "MANA-USDT-SWAP", "GALA-USDT-SWAP", "ENJ-USDT-SWAP",
            "BAT-USDT-SWAP", "COMP-USDT-SWAP", "MKR-USDT-SWAP", "AAVE-USDT-SWAP",
            "SNX-USDT-SWAP", "CRV-USDT-SWAP", "UNI-USDT-SWAP", "SUSHI-USDT-SWAP"
        ]

    def get_1m_candles(self, symbol: str, limit: int = 50) -> pd.DataFrame:
        """Получение 1-минутных данных для быстрого скальпинга"""
        try:
            url = f"{self.base_url}/api/v5/market/candles"
            params = {
                'instId': symbol,
                'bar': '1m',
                'limit': limit
            }

            response = requests.get(url, params=params, timeout=5)
            data = response.json()

            if data['code'] == '0' and data['data']:
                candles = data['data']
                candles.reverse()

                df_data = []
                for candle in candles:
                    df_data.append({
                        'Timestamp': datetime.fromtimestamp(int(candle[0]) / 1000),
                        'Open': float(candle[1]),
                        'High': float(candle[2]),
                        'Low': float(candle[3]),
                        'Close': float(candle[4]),
                        'Volume': float(candle[5]),
                        'VolumeCcy': float(candle[6])
                    })

                df = pd.DataFrame(df_data)
                df.set_index('Timestamp', inplace=True)
                return df
            else:
                return None

        except Exception as e:
            return None

    def get_current_price(self, symbol: str) -> float:
        """Получение текущей цены в реальном времени"""
        try:
            url = f"{self.base_url}/api/v5/market/ticker"
            params = {'instId': symbol}

            response = requests.get(url, params=params, timeout=3)
            data = response.json()

            if data['code'] == '0' and data['data']:
                return float(data['data'][0]['last'])
            return None
        except Exception as e:
            print(f"❌ Ошибка получения текущей цены для {symbol}: {e}")
            return None

    def get_btc_trend_analysis(self) -> Dict[str, Any]:
        """Анализ тренда Bitcoin для фильтрации сигналов"""
        try:
            # Получаем данные BTC на нескольких таймфреймах
            btc_data_1m = self.get_1m_candles("BTC-USDT-SWAP", 30)
            btc_data_5m = self.get_1m_candles("BTC-USDT-SWAP", 100)  # Используем 1m данные, но больше периодов

            if btc_data_1m is None or btc_data_5m is None:
                return {'trend': 'neutral', 'strength': 0, 'direction': 0}

            # Анализ на 1m таймфрейме (быстрый тренд)
            btc_data_1m['ema_5'] = btc_data_1m['Close'].ewm(span=5).mean()
            btc_data_1m['ema_10'] = btc_data_1m['Close'].ewm(span=10).mean()
            btc_data_1m['rsi'] = self._calculate_rsi(btc_data_1m['Close'], 6)

            # Анализ на 5m таймфрейме (среднесрочный тренд)
            btc_data_5m['ema_10'] = btc_data_5m['Close'].ewm(span=10).mean()
            btc_data_5m['ema_20'] = btc_data_5m['Close'].ewm(span=20).mean()

            current_1m = btc_data_1m.iloc[-1]
            current_5m = btc_data_5m.iloc[-1]

            # Оценка тренда по нескольким факторам
            trend_score = 0

            # 1. EMA пересечения
            if current_1m['ema_5'] > current_1m['ema_10']:
                trend_score += 1
            else:
                trend_score -= 1

            if current_5m['ema_10'] > current_5m['ema_20']:
                trend_score += 2
            else:
                trend_score -= 2

            # 2. RSI анализ
            if current_1m['rsi'] > 60:
                trend_score += 1
            elif current_1m['rsi'] < 40:
                trend_score -= 1

            # 3. Ценовое движение
            price_change_5m = ((btc_data_5m['Close'].iloc[-1] - btc_data_5m['Close'].iloc[-5]) /
                             btc_data_5m['Close'].iloc[-5]) * 100
            if price_change_5m > 0.3:
                trend_score += 1
            elif price_change_5m < -0.3:
                trend_score -= 1

            # Определение тренда
            if trend_score >= 3:
                return {'trend': 'strong_bullish', 'strength': abs(trend_score), 'direction': 1}
            elif trend_score >= 1:
                return {'trend': 'bullish', 'strength': abs(trend_score), 'direction': 1}
            elif trend_score <= -3:
                return {'trend': 'strong_bearish', 'strength': abs(trend_score), 'direction': -1}
            elif trend_score <= -1:
                return {'trend': 'bearish', 'strength': abs(trend_score), 'direction': -1}
            else:
                return {'trend': 'neutral', 'strength': 0, 'direction': 0}

        except Exception as e:
            print(f"❌ Ошибка анализа тренда BTC: {e}")
            return {'trend': 'neutral', 'strength': 0, 'direction': 0}

    def _calculate_rsi(self, prices: pd.Series, period: int = 14) -> pd.Series:
        """Расчет RSI"""
        delta = prices.diff()
        gain = (delta.where(delta > 0, 0)).ewm(span=period).mean()
        loss = (-delta.where(delta < 0, 0)).ewm(span=period).mean()
        rs = gain / loss
        return 100 - (100 / (1 + rs))

class ScalpingSignalGenerator:
    def __init__(self):
        # Агрессивные параметры для быстрого скальпинга
        self.parameters = {
            'ema_fast': 3,      # Очень быстрая EMA
            'ema_slow': 8,      # Быстрая EMA
            'rsi_period': 6,    # Короткий RSI
            'volume_ma': 5,     # Короткий объем
            'atr_period': 5     # Короткий ATR
        }

    def calculate_ultra_fast_indicators(self, df: pd.DataFrame) -> pd.DataFrame:
        """Расчет ультра-быстрых индикаторов для 1m скальпинга"""
        p = self.parameters

        # Супер-быстрые EMA
        df['ema_fast'] = df['Close'].ewm(span=p['ema_fast']).mean()
        df['ema_slow'] = df['Close'].ewm(span=p['ema_slow']).mean()

        # Быстрый RSI
        delta = df['Close'].diff()
        gain = (delta.where(delta > 0, 0)).ewm(span=p['rsi_period']).mean()
        loss = (-delta.where(delta < 0, 0)).ewm(span=p['rsi_period']).mean()
        rs = gain / loss
        df['rsi'] = 100 - (100 / (1 + rs))

        # VWAP для скальпинга
        typical_price = (df['High'] + df['Low'] + df['Close']) / 3
        df['vwap'] = (typical_price * df['Volume']).cumsum() / df['Volume'].cumsum()

        # Быстрый ATR
        high_low = df['High'] - df['Low']
        high_close = np.abs(df['High'] - df['Close'].shift())
        low_close = np.abs(df['Low'] - df['Close'].shift())
        true_range = np.maximum(np.maximum(high_low, high_close), low_close)
        df['atr'] = true_range.ewm(span=p['atr_period']).mean()

        # Объем для скальпинга
        df['volume_ma'] = df['Volume'].ewm(span=p['volume_ma']).mean()
        df['volume_ratio'] = df['Volume'] / df['volume_ma']

        # Моментум индикаторы
        df['price_change_1m'] = df['Close'].pct_change(1) * 100
        df['price_change_3m'] = df['Close'].pct_change(3) * 100

        # MACD для скальпинга
        df['macd_fast'] = df['Close'].ewm(span=4).mean()
        df['macd_slow'] = df['Close'].ewm(span=10).mean()
        df['macd'] = df['macd_fast'] - df['macd_slow']
        df['macd_signal'] = df['macd'].ewm(span=3).mean()

        return df

    def generate_scalping_signals(self, df: pd.DataFrame, symbol: str, btc_trend: Dict[str, Any], current_price: float) -> List[Dict[str, Any]]:
        """Генерация быстрых скальпинговых сигналов с учетом тренда BTC и проверкой актуальности"""
        if df is None or len(df) < 10:
            return []

        signals = []

        # Используем только ПОСЛЕДНЮЮ свечу для актуальности
        current = df.iloc[-1]
        prev = df.iloc[-2]

        # УВЕЛИЧЕНА проверка актуальности данных - разница между текущей ценой и ценой закрытия последней свечи
        if current_price:
            price_diff_pct = abs(current_price - current['Close']) / current['Close'] * 100
            # УВЕЛИЧЕН порог с 0.5% до 1.5% для большей актуальности
            if price_diff_pct > 1.5:  # Если разница более 1.5%, данные устарели
                print(f"⚠️ Данные устарели для {symbol}: разница {price_diff_pct:.2f}%")
                return []

        # Быстрые условия для скальпинга
        buy_conditions = self._check_buy_conditions(current, prev)
        sell_conditions = self._check_sell_conditions(current, prev)

        # Корректировка силы сигнала на основе тренда BTC
        buy_conditions = self._adjust_signal_for_btc_trend(buy_conditions, btc_trend, 'LONG')
        sell_conditions = self._adjust_signal_for_btc_trend(sell_conditions, btc_trend, 'SHORT')

        # Используем актуальную цену для расчетов
        entry_price = current_price if current_price else current['Close']

        if buy_conditions['score'] >= 2.0:  # Понижен порог после корректировки BTC
            signal = {
                'symbol': symbol,
                'position_type': 'LONG',
                'entry_price': float(entry_price),
                'timestamp': current.name,
                'strength': min(5, int(buy_conditions['score'])),
                'stop_loss': float(entry_price - current['atr'] * 0.8),
                'take_profit': float(entry_price + current['atr'] * 1.2),
                'indicators': ', '.join(buy_conditions['indicators']),
                'timeframe': '1m',
                'leverage': 10,
                'rr_ratio': '1:1.5',
                'btc_trend': btc_trend['trend'],
                'btc_aligned': buy_conditions['btc_aligned'],
                'data_freshness': price_diff_pct if current_price else 999
            }
            signals.append(signal)

        elif sell_conditions['score'] >= 2.0:
            signal = {
                'symbol': symbol,
                'position_type': 'SHORT',
                'entry_price': float(entry_price),
                'timestamp': current.name,
                'strength': min(5, int(sell_conditions['score'])),
                'stop_loss': float(entry_price + current['atr'] * 0.8),
                'take_profit': float(entry_price - current['atr'] * 1.2),
                'indicators': ', '.join(sell_conditions['indicators']),
                'timeframe': '1m',
                'leverage': 10,
                'rr_ratio': '1:1.5',
                'btc_trend': btc_trend['trend'],
                'btc_aligned': sell_conditions['btc_aligned'],
                'data_freshness': price_diff_pct if current_price else 999
            }
            signals.append(signal)

        return signals

    def _check_buy_conditions(self, current, prev) -> Dict[str, Any]:
        """Проверка условий для LONG позиции"""
        score = 0
        indicators = []

        # 1. EMA пересечение
        if current['ema_fast'] > current['ema_slow'] and prev['ema_fast'] <= prev['ema_slow']:
            score += 1.5
            indicators.append("EMA")

        # 2. RSI перепроданность
        if current['rsi'] < 25:
            score += 1.2
            indicators.append("RSI")
        elif current['rsi'] < 35:
            score += 0.8
            indicators.append("RSI")

        # 3. Цена выше VWAP
        if current['Close'] > current['vwap']:
            score += 0.5
            indicators.append("VWAP")

        # 4. Объемный спрейк
        if current['volume_ratio'] > 2.0:
            score += 1.0
            indicators.append("VOL")
        elif current['volume_ratio'] > 1.5:
            score += 0.5
            indicators.append("VOL")

        # 5. Моментум
        if current['price_change_1m'] > 0.1:
            score += 0.3
            indicators.append("MOM")

        # 6. MACD
        if current['macd'] > current['macd_signal'] and prev['macd'] <= prev['macd_signal']:
            score += 1.0
            indicators.append("MACD")

        return {'score': score, 'indicators': indicators, 'btc_aligned': True}

    def _check_sell_conditions(self, current, prev) -> Dict[str, Any]:
        """Проверка условий для SHORT позиции"""
        score = 0
        indicators = []

        # 1. EMA пересечение
        if current['ema_fast'] < current['ema_slow'] and prev['ema_fast'] >= prev['ema_slow']:
            score += 1.5
            indicators.append("EMA")

        # 2. RSI перекупленность
        if current['rsi'] > 75:
            score += 1.2
            indicators.append("RSI")
        elif current['rsi'] > 65:
            score += 0.8
            indicators.append("RSI")

        # 3. Цена ниже VWAP
        if current['Close'] < current['vwap']:
            score += 0.5
            indicators.append("VWAP")

        # 4. Объемный спрейк
        if current['volume_ratio'] > 2.0:
            score += 1.0
            indicators.append("VOL")
        elif current['volume_ratio'] > 1.5:
            score += 0.5
            indicators.append("VOL")

        # 5. Моментум
        if current['price_change_1m'] < -0.1:
            score += 0.3
            indicators.append("MOM")

        # 6. MACD
        if current['macd'] < current['macd_signal'] and prev['macd'] >= prev['macd_signal']:
            score += 1.0
            indicators.append("MACD")

        return {'score': score, 'indicators': indicators, 'btc_aligned': True}

    def _adjust_signal_for_btc_trend(self, conditions: Dict[str, Any], btc_trend: Dict[str, Any], signal_type: str) -> Dict[str, Any]:
        """Корректировка силы сигнала на основе тренда BTC"""
        btc_direction = btc_trend['direction']
        btc_strength = btc_trend['strength']

        # Определяем согласованность с трендом BTC
        is_aligned = (
            (btc_direction == 1 and signal_type == 'LONG') or
            (btc_direction == -1 and signal_type == 'SHORT') or
            btc_direction == 0  # Нейтральный тренд BTC
        )

        conditions['btc_aligned'] = is_aligned

        if is_aligned:
            # Усиливаем сигналы, идущие в направлении тренда BTC
            if btc_strength >= 3:  # Сильный тренд BTC
                conditions['score'] *= 1.5
            elif btc_strength >= 1:  # Умеренный тренд BTC
                conditions['score'] *= 1.2
            # Добавляем индикатор BTC тренда
            if 'BTC' not in conditions['indicators']:
                conditions['indicators'] += ", BTC-TREND"
        else:
            # Ослабляем сигналы, идущие против тренда BTC
            if btc_strength >= 3:  # Сильный тренд BTC
                conditions['score'] *= 0.3  # Сильно ослабляем
            elif btc_strength >= 1:  # Умеренный тренд BTC
                conditions['score'] *= 0.6  # Ослабляем
            # Добавляем предупреждение
            if 'BTC-WARN' not in conditions['indicators']:
                conditions['indicators'] += ", ⚠️-BTC"

        return conditions

class TelegramBot:
    def __init__(self, bot_token: str, chat_id: str):
        self.bot_token = bot_token
        self.chat_id = chat_id
        self.base_url = f"https://api.telegram.org/bot{bot_token}/"

    def send_scalping_signal(self, signal: Dict[str, Any]) -> bool:
        """Отправка скальпингового сигнала с учетом тренда BTC"""
        emoji = "🟢" if signal['position_type'] == 'LONG' else "🔴"

        # Определяем срочность на основе согласованности с BTC
        if signal['btc_aligned']:
            if signal['strength'] >= 4:
                urgency = "🚨🚨"
            else:
                urgency = "🚨"
        else:
            urgency = "⚠️"

        # Определяем эмодзи для тренда BTC
        btc_emoji = "🟢" if "bull" in signal['btc_trend'] else "🔴" if "bear" in signal['btc_trend'] else "⚪"
        alignment_emoji = "✅" if signal['btc_aligned'] else "❌"

        # ОБНОВЛЕНА информация о свежести данных с увеличенными порогами
        freshness = "🟢 СВЕЖИЕ" if signal.get('data_freshness', 999) < 0.8 else "🟡 НОРМА" if signal.get('data_freshness', 999) < 1.5 else "🔴 УСТАРЕЛИ"

        message = f"""
{urgency} {emoji} <b>SCALPING SIGNAL</b> {emoji} {urgency}

<b>🎯 {signal['symbol']}</b>
<b>Позиция:</b> {signal['position_type']} | <b>Сила:</b> {signal['strength']}/5
<b>Плечо:</b> {signal['leverage']}x | <b>Таймфрейм:</b> {signal['timeframe']}

<b>💰 Цена входа:</b> ${signal['entry_price']:.4f}
<b>🛡️ Stop-Loss:</b> ${signal['stop_loss']:.4f}
<b>🎯 Take-Profit:</b> ${signal['take_profit']:.4f}
<b>📈 R/R:</b> {signal['rr_ratio']}

<b>📊 Индикаторы:</b> {signal['indicators']}

<b>₿ Тренд BTC:</b> {btc_emoji} {signal['btc_trend'].replace('_', ' ').upper()}
<b>📊 Согласованность:</b> {alignment_emoji} {'СОГЛАСОВАН' if signal['btc_aligned'] else 'ПРОТИВ ТРЕНДА'}
<b>📡 Данные:</b> {freshness} ({signal.get('data_freshness', 0):.2f}%)

<b>⏰ Время:</b> {datetime.now().strftime('%H:%M:%S')}
<b>⚡ Тип:</b> Скальпинг (1-5 минут)
        """

        return self._send_message(message)

    def send_scan_summary(self, total_symbols: int, total_signals: int, aligned_signals: int, btc_trend: str, scan_time: float):
        """Отправка сводки сканирования"""
        alignment_rate = (aligned_signals / total_signals * 100) if total_signals > 0 else 0

        message = f"""
<b>📊 SCAN SUMMARY</b>

<b>🔍 Просканировано пар:</b> {total_symbols}
<b>🎯 Найдено сигналов:</b> {total_signals}
<b>✅ Согласованных с BTC:</b> {aligned_signals} ({alignment_rate:.1f}%)
<b>₿ Тренд BTC:</b> {btc_trend.replace('_', ' ').upper()}
<b>⏱️ Время сканирования:</b> {scan_time:.2f} сек

<b>⏰ Следующее сканирование через 15 секунд</b>
        """

        return self._send_message(message)

    def _send_message(self, text: str) -> bool:
        """Отправка сообщения в Telegram"""
        url = self.base_url + "sendMessage"
        payload = {
            'chat_id': self.chat_id,
            'text': text,
            'parse_mode': 'HTML'
        }
        try:
            response = requests.post(url, data=payload, timeout=5)
            return response.status_code == 200
        except:
            return False

class AllFuturesScalpingScanner:
    def __init__(self, telegram_bot: TelegramBot):
        self.scanner = OKXFuturesScanner()
        self.signal_generator = ScalpingSignalGenerator()
        self.telegram_bot = telegram_bot
        self.processed_signals = set()

        # Загружаем все символы при инициализации
        self.all_symbols = self.scanner.get_all_usdt_futures_symbols()
        print(f"✅ Загружено {len(self.all_symbols)} USDT фьючерсных пар для сканирования")

    def is_signal_valid(self, signal: Dict[str, Any], current_price: float) -> bool:
        """Проверяет актуальность сигнала перед отправкой"""
        if current_price is None:
            return False

        # УВЕЛИЧЕНА проверка разницы между текущей ценой и ценой входа
        price_diff = abs(current_price - signal['entry_price']) / signal['entry_price'] * 100

        # УВЕЛИЧЕН порог с 0.3% до 1.0% - сигнал считается устаревшим только при изменении более чем на 1.0%
        if price_diff > 1.0:
            print(f"⚠️ Сигнал устарел для {signal['symbol']}: разница {price_diff:.2f}%")
            return False

        # Проверяем, не достигнуты ли уже уровни тейк-профита или стоп-лосса
        if signal['position_type'] == 'LONG':
            if current_price >= signal['take_profit']:
                print(f"⚠️ Тейк-профит уже достигнут для {signal['symbol']}")
                return False
            if current_price <= signal['stop_loss']:
                print(f"⚠️ Стоп-лосс уже достигнут для {signal['symbol']}")
                return False
        else:  # SHORT
            if current_price <= signal['take_profit']:
                print(f"⚠️ Тейк-профит уже достигнут для {signal['symbol']}")
                return False
            if current_price >= signal['stop_loss']:
                print(f"⚠️ Стоп-лосс уже достигнут для {signal['symbol']}")
                return False

        return True

    def scan_symbol(self, symbol: str, btc_trend: Dict[str, Any]) -> List[Dict[str, Any]]:
        """Сканирование одной пары на наличие сигналов с учетом тренда BTC"""
        try:
            # Получаем 1-минутные данные
            data = self.scanner.get_1m_candles(symbol, 30)

            if data is None or len(data) < 10:
                return []

            # Получаем текущую цену в реальном времени
            current_price = self.scanner.get_current_price(symbol)

            # Рассчитываем индикаторы
            data = self.signal_generator.calculate_ultra_fast_indicators(data)
            data = data.dropna()

            if len(data) < 5:
                return []

            # Генерируем сигналы с учетом тренда BTC и текущей цены
            signals = self.signal_generator.generate_scalping_signals(data, symbol, btc_trend, current_price)

            # Фильтруем устаревшие сигналы
            valid_signals = []
            for signal in signals:
                if self.is_signal_valid(signal, current_price):
                    valid_signals.append(signal)

            return valid_signals

        except Exception as e:
            print(f"❌ Ошибка сканирования {symbol}: {e}")
            return []

    def run_scan(self) -> List[Dict[str, Any]]:
        """Запуск сканирования всех пар с анализом тренда BTC"""
        print(f"\n🔍 Сканирование {len(self.all_symbols)} пар...")
        start_time = time.time()

        # Получаем тренд BTC перед сканированием
        btc_trend = self.scanner.get_btc_trend_analysis()
        print(f"₿ Тренд BTC: {btc_trend['trend']} (сила: {btc_trend['strength']})")

        all_signals = []

        # Используем многопоточность для ускорения сканирования
        with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
            future_to_symbol = {
                executor.submit(self.scan_symbol, symbol, btc_trend): symbol
                for symbol in self.all_symbols
            }

            completed = 0
            for future in concurrent.futures.as_completed(future_to_symbol):
                symbol = future_to_symbol[future]
                try:
                    signals = future.result()
                    if signals:
                        all_signals.extend(signals)
                    completed += 1

                    if completed % 20 == 0:
                        print(f"📊 Обработано {completed}/{len(self.all_symbols)} пар...")

                except Exception as e:
                    completed += 1

        scan_time = time.time() - start_time

        # Сортируем сигналы по силе и согласованности с BTC
        all_signals.sort(key=lambda x: (x['btc_aligned'], x['strength']), reverse=True)

        # Считаем согласованные сигналы
        aligned_signals = sum(1 for signal in all_signals if signal['btc_aligned'])

        print(f"✅ Сканирование завершено за {scan_time:.2f} сек")
        print(f"🎯 Найдено сигналов: {len(all_signals)}")
        print(f"✅ Согласованных с BTC: {aligned_signals}")

        # Отправляем сводку
        self.telegram_bot.send_scan_summary(
            len(self.all_symbols), len(all_signals), aligned_signals,
            btc_trend['trend'], scan_time
        )

        return all_signals

    def send_signals(self, signals: List[Dict[str, Any]], max_signals: int = 15):
        """Отправка сигналов в Telegram с приоритетом согласованных с BTC"""
        sent_count = 0

        # Сначала отправляем сигналы, согласованные с BTC
        aligned_signals = [s for s in signals if s['btc_aligned']]
        other_signals = [s for s in signals if not s['btc_aligned']]

        # Ограничиваем количество каждого типа
        max_aligned = min(10, max_signals)
        max_other = min(5, max_signals - max_aligned)

        priority_signals = aligned_signals[:max_aligned] + other_signals[:max_other]

        for signal in priority_signals:
            # Создаем уникальный ключ сигнала
            signal_key = f"{signal['symbol']}_{signal['position_type']}_{signal['timestamp'].strftime('%H%M%S')}"

            # Проверяем, не отправляли ли уже этот сигнал
            if signal_key not in self.processed_signals:
                # Двойная проверка актуальности перед отправкой
                current_price = self.scanner.get_current_price(signal['symbol'])
                if current_price and self.is_signal_valid(signal, current_price):
                    if self.telegram_bot.send_scalping_signal(signal):
                        sent_count += 1
                        self.processed_signals.add(signal_key)

                        status = "✅" if signal['btc_aligned'] else "⚠️"
                        print(f"{status} Отправлен сигнал для {signal['symbol']} (сила: {signal['strength']})")

                        # Небольшая пауза между отправками
                        time.sleep(0.5)
                else:
                    print(f"❌ Сигнал устарел перед отправкой для {signal['symbol']}")

        # Очищаем старые сигналы (сохраняем только последние 100)
        if len(self.processed_signals) > 100:
            self.processed_signals = set(list(self.processed_signals)[-100:])

        return sent_count

# ОСНОВНОЙ ЗАПУСК
if __name__ == "__main__":
    # === НАСТРОЙКИ ===
    BOT_TOKEN = "8294892098:AAFX0Zzq9yN1on6UlID8f7vzif4dWR_7uWs"
    CHAT_ID = "381202205"

    SCAN_INTERVAL = 15  # Уменьшено до 15 секунд для большей актуальности
    MAX_SIGNALS_PER_SCAN = 15  # Максимум сигналов за сканирование

    print("🚀 ЗАПУСК SCALPING SCANNER ДЛЯ ВСЕХ USDT ФЬЮЧЕРСОВ С УЧЕТОМ BTC")
    print("=" * 60)
    print("⚡ ВЕРСИЯ 2.1 - УВЕЛИЧЕНА АКТУАЛЬНОСТЬ СИГНАЛОВ")
    print("=" * 60)

    # Инициализация бота
    telegram_bot = TelegramBot(BOT_TOKEN, CHAT_ID)

    # Тестовое сообщение
    telegram_bot._send_message("🤖 <b>BTC-Aligned Scalping Scanner v2.1 активирован!</b>\n\n⚡ <b>Увеличена актуальность сигналов</b>\n📊 Пороги устаревания: 1.0%-1.5%\n⏱️ Интервал сканирования: 15 секунд")

    # Инициализация сканера
    scanner = AllFuturesScalpingScanner(telegram_bot)

    print("✅ Сканер готов к работе!")
    print(f"🔍 Будут сканироваться {len(scanner.all_symbols)} пар каждые {SCAN_INTERVAL} секунд")
    print("₿ Сигналы будут корректироваться в соответствии с трендом Bitcoin")
    print("⚡ УВЕЛИЧЕНА актуальность сигналов (порог устаревания: 1.0%-1.5%)")

    cycle_count = 0

    while True:
        try:
            cycle_count += 1
            print(f"\n🔄 ЦИКЛ СКАНИРОВАНИЯ #{cycle_count}")
            print("=" * 40)

            # Запускаем сканирование
            signals = scanner.run_scan()

            # Отправляем сигналы
            if signals:
                sent_count = scanner.send_signals(signals, MAX_SIGNALS_PER_SCAN)
                print(f"📤 Отправлено сигналов: {sent_count}/{len(signals)}")
            else:
                print("➖ Сигналов не найдено")

            # Ожидание следующего сканирования
            print(f"💤 Ожидание {SCAN_INTERVAL} секунд...")
            time.sleep(SCAN_INTERVAL)

        except KeyboardInterrupt:
            print("\n⏹️ Остановка сканера...")
            telegram_bot._send_message("🛑 <b>Scalping Scanner остановлен</b>")
            break
        except Exception as e:
            print(f"❌ Критическая ошибка: {e}")
            print("💤 Повтор через 30 секунд...")
            time.sleep(30)

🚀 ЗАПУСК SCALPING SCANNER ДЛЯ ВСЕХ USDT ФЬЮЧЕРСОВ С УЧЕТОМ BTC
⚡ ВЕРСИЯ 2.1 - УВЕЛИЧЕНА АКТУАЛЬНОСТЬ СИГНАЛОВ
🔍 Получение списка всех USDT фьючерсных пар...
✅ Найдено 242 USDT фьючерсных пар
✅ Загружено 242 USDT фьючерсных пар для сканирования
✅ Сканер готов к работе!
🔍 Будут сканироваться 242 пар каждые 15 секунд
₿ Сигналы будут корректироваться в соответствии с трендом Bitcoin
⚡ УВЕЛИЧЕНА актуальность сигналов (порог устаревания: 1.0%-1.5%)

🔄 ЦИКЛ СКАНИРОВАНИЯ #1

🔍 Сканирование 242 пар...
₿ Тренд BTC: strong_bullish (сила: 4)
📊 Обработано 20/242 пар...
📊 Обработано 40/242 пар...
📊 Обработано 60/242 пар...
📊 Обработано 80/242 пар...
📊 Обработано 100/242 пар...
📊 Обработано 120/242 пар...
📊 Обработано 140/242 пар...
📊 Обработано 160/242 пар...
📊 Обработано 180/242 пар...
📊 Обработано 200/242 пар...
📊 Обработано 220/242 пар...
📊 Обработано 240/242 пар...
✅ Сканирование завершено за 12.59 сек
🎯 Найдено сигналов: 34
✅ Согласованных с BTC: 34
❌ Сигнал устарел перед отправкой для COOKIE-U